<a href="https://colab.research.google.com/github/JaroslavHolecek/Teaching/blob/master/JupyterNotebook/SQL/Vlastnosti_atributu_zadani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vlastnosti atributů
Jak bylo již zmíněno v souboru [Create_alter_delete_zadani.ipynb](https://github.com/JaroslavHolecek/Teaching/blob/master/JupyterNotebook/SQL/Create_alter_delete_zadani.ipynb), tak sloupečky tabulek mohou mít nejrůznější vlastnosti. A na některé z nich se podíváme.

# Primary key
Primary key, neboli vlastní klíč slouží k jednoznačné identifakici řádku v tabulce. Ve sloupečku označeném jako "PRIMARY KEY" se nesmí žádná hodnota opakovat dvakrát, toto za nás pohlídá sama databáze.
Při vypisování hodnot z databáze se nám pak vrátí buď <strong>jeden</strong> řádek, nebo žádný.
Jako vlastní klíč můžeme označit i více atributů dohromady. V tabulce se potom nebude opakovat řádek, který má kombinaci těchto hodnot stejnou.

## Ukázka jednoho vlastního klíče v tabulce
Spolu s vlastním klíčem se často na atribut používá funkce AUTO_INCREMENT, to nám zajistí že si nemusíme pamatovat v tomto případě poslední přidělené ID.
Téměř vždy budeme jako vlastní klíč používat atribut ID.

In [ ]:
!pip install mysql-connector

In [8]:
import mysql.connector

mydb = mysql.connector.connect(
    host = ""
    ,user = ""
    ,password = ""
    ,database = ""
)
mycursor = mydb.cursor()

mycursor.execute("""
CREATE TABLE Automobil(
    id int PRIMARY KEY AUTO_INCREMENT, 
    jmeno text,
    prijmeni text
)""")

mydb.commit()

mycursor.close()
mydb.close()


## Vlastní klíč přes dva (či více) sloupečky(ů)
Abychom nastavili omezení přes více sloupečků, použijeme CONSTRAINT (omezení, podmínku). Toto omezení pojmenujeme (název se nám zobrazí, např. pokud nastane chyba právě v tomto omezení -> pokusíme se např. zadat druhého člověka se stejným jménem i příjmením) a nastavíme co má omezovat. Obecně tato funkce vypadá takto:

<code>CONSTRAINT Nazev PRIMARY KEY (atribut1, atribut2)</code>

In [11]:
mydb = mysql.connector.connect(
    host = ""
    ,user = ""
    ,password = ""
    ,database = ""
)
mycursor = mydb.cursor()

# mycursor.execute("DROP TABLE Automobil")
mycursor.execute("""CREATE TABLE Automobil(
    id int, 
    jmeno char(10),
    prijmeni char(20),
    CONSTRAINT PK_jmeno_prijmeni PRIMARY KEY (jmeno, prijmeni)
);""")

mydb.commit()

mycursor.close()
mydb.close()

# Foreign key
Pomocí "cizího klíče" řekneme databázi, že hodnoty uložené v tomto sloupečku jsou převzaté z jiného sloupečku v jiné tabulce.
SŘBD nám bude hlídat zda v něm nemáme hodnotu, která v druhé tabulce neexistuje. Také můžeme nastavit, co se má stát, když některou hodnotu chceme smazat a máme na ni navázanou jinou hodnotu (FOREIGN KEY) v jiné tabulce - nepůjde smazat / smaže se řádek v obou tabulkách (ON DELETE CASCADE).

In [19]:
mydb = mysql.connector.connect(
    host = ""
    ,user = ""
    ,password = ""
    ,database = ""
)
mycursor = mydb.cursor()

# mycursor.execute("DROP TABLE Clovek")
mycursor.execute("DROP TABLE Kvalifikace")
mycursor.execute("""CREATE TABLE Kvalifikace(
    id int PRIMARY KEY AUTO_INCREMENT, 
    oznaceni text,
    popis text
);""")
mydb.commit()


mycursor.execute("""CREATE TABLE Clovek(
    id int PRIMARY KEY AUTO_INCREMENT, 
    jmeno text,
    prijmeni text,
    datum_narozeni date,
    kvalifikace int,
    FOREIGN KEY (kvalifikace) REFERENCES Kvalifikace(id)
);""")

mydb.commit()

mycursor.close()
mydb.close()

Atribut <code>kvalifikace</code> jsme nastavili jako Cizí klíč, ve kterém jsou hodnoty ze sloupce <code>id</code> z tabulky <code>Kvalifikace</code>.
Vlastní klíč "id" v tabulce Kvalifikace volíme záměrně - zapsat do sloupečku "kvalifikace" můžeme jen jednu hodnotu a proto musíme zajistit, aby byla možná jen jedna hodnota ve zdrojovém sloupečku "id".

# NOT NULL
Označuje atribut, ve kterém nemůže být uložena hodnota NULL - tedy prázdno - tedy vždy musíme hodnotu vyplnit.

# AUTO_INCREMENT
V tomto atributu se automaticky doplní hodnota. Databáze zjistí nejvyšší hodnotu v tomto sloupečku, přičte 1 a výsledek vloží na nové místo.
Typicky se dává na atribut s názvem "id".

# UNIQUE
V takto označeném atributu se nesmí žádná hodnota opakovat.
Lze ji nastavit i na více sloupečků. Potom se nemůže opakovat stejná tato dvojice, ale hodnota v každém sloupčku se opakovat může. Takovou vlastnost napíšeme pomocí CONSTRAINT - vysvětlena níže.

# DEFAULT
Pokud nastavíme atributu DEFAULT hodnotu a poté při vkládání záznamu nezapíšeme, jaká hodnota se má do tohoto sloupečku vložit, vloží se tato nastavená hodnota.

# CONSTRAINT
Tuto vlastnost využijeme ve chvíli kdy:
- Do tabulky chceme vložit nějaké omezení (např. NOT NULL, UNIQUE) až poté, co jsme ji již vytvořili a nechceme celou tabulku vytvářet znovu.
- Chceme nastavit omezení přes více sloupečků (nebude se moct 2x objevit kombinace sloupečků, hodnoty v samotných sloupčcích se opakovat mohou).
- Chceme omezení pojmenovat (aby se nám přehledně zobrazovalo např. v chybových hláškách).

## Zápis přímo do tabulky

In [23]:
mydb = mysql.connector.connect(
    host = ""
    ,user = ""
    ,password = ""
    ,database = ""
)
mycursor = mydb.cursor()

# mycursor.execute("DROP TABLE Clovek")
mycursor.execute("""CREATE TABLE Clovek( 
    jmeno char(10) NOT NULL,
    prijmeni char(20) NOT NULL,
    datum_narozeni date,
    kvalifikace text,
    CONSTRAINT PK_Clovek PRIMARY KEY (jmeno, prijmeni),
    CONSTRAINT nechceme_dvojcata UNIQUE (datum_narozeni)
);""")

mydb.commit()

mycursor.close()
mydb.close()

## Zápis až po vytvoření tabulky

In [26]:
mydb = mysql.connector.connect(
    host = ""
    ,user = ""
    ,password = ""
    ,database = ""
)
mycursor = mydb.cursor()

mycursor.execute("DROP TABLE Clovek")
mycursor.execute("""CREATE TABLE Clovek( 
    jmeno char(10) NOT NULL,
    prijmeni char(20) NOT NULL,
    datum_narozeni date,
    kvalifikace text
);""")
mydb.commit()

mycursor.execute("""ALTER TABLE Clovek ADD CONSTRAINT PK_Clovek PRIMARY KEY (jmeno, prijmeni);""")
mydb.commit()

mycursor.execute("""ALTER TABLE Clovek ADD CONSTRAINT nechceme_dvojcata UNIQUE (datum_narozeni)""")


mycursor.close()
mydb.close()

# Cvičeni
Bude následovat pouze jedno cvičení.

Připojte se do databáze a vytvořte tabulku Auto s atributy a vlastnostmi atributu:
- id(int, hodnota se sama vyplní, vlastní klíč)
- znacka(char(20), nesmí být prázdné)
- spz(char(7), nesmí být prázdné, musí být jedinečná hodnota)

Vytvořte tabulku Ridic s atributy a vlastnostmi atributu:
- id(int, hodnota se sama vyplní, vlastní klíč)
- jmeno(char(10), nesmí být prázdné)
- prijmeni(char(20), nesmí být prázdné)
- auto(int, cizí klíč odkazující na atribut "id" tabulky Auto)
- Jedna kombinace jmena a prijmeni se nesmí objevit vícekrát, hodnoty samotných atributech se opakovat mohou, tuto kombinaci pojmenujte unikatni_jmeno





In [ ]:
# zde přijde váš kód ->


# >>> tuto část neměnit ! 
mycursor.execute("""DESCRIBE Auto""")
table = str(mycursor.fetchall())
expected = "[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('znacka', 'char(20)', 'NO', '', None, ''), ('spz', 'char(7)', 'NO', 'UNI', None, '')]"
if table == expected:
    print("Tabulka Auto vytvořena správně.")
else:
  print("Tabulka Auto není dle zadání.\nVaše tabulka:")
  print(table)
  print("Očekávaná tabulka:")
  print(expected)
  print("Výpis se liší na pozicích:", [ i for i in range(min(len(table),len(expected))) if table[i] != expected[i] ])


mycursor.execute("""DESCRIBE Ridic""")
table = str(mycursor.fetchall())
expected = "[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('jmeno', 'char(10)', 'NO', '', None, ''), ('prijmeni', 'char(20)', 'NO', '', None, ''), ('auto', 'int(11)', 'YES', 'MUL', None, '')]"
if table == expected:
    print("Tabulka Ridic vytvořena správně.")
else:
  print("Tabulka Ridic není dle zadání.\nVaše tabulka:")
  print(table)
  print("Očekávaná tabulka:")
  print(expected)
  print("Výpis se liší na pozicích:", [ i for i in range(min(len(table),len(expected))) if table[i] != expected[i] ])

mycursor.execute("""DROP TABLE Ridic""")
mycursor.execute("""DROP TABLE Auto""")
mydb.commit()

mycursor.close()
mydb.close()